### Extraindo o arquivo .zip

In [1]:
import requests
import zipfile

In [2]:
url = "https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2021.zip"

In [3]:
r=requests.get(url, verify=False)

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
with open("r.zip","wb") as f:
    f.write(r.content)

In [9]:
with zipfile.ZipFile("r.zip","r") as zip_ref:
    zip_ref.extractall("./data")

### import sparkssesion

In [23]:
from pyspark.sql import SparkSession

In [24]:
spark = SparkSession \
.builder \
.config("spark.sql.repl.eagerEval.enabled", "True") \
.config("spark.sql.repl.eagerEval.maxNumRows", "10") \
.getOrCreate()

In [27]:
df = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2021/dados/MICRODADOS_CADASTRO_IES_2021.CSV", sep=';', header=True, inferSchema=True)

In [28]:
for col in df.columns:
    df = df.withColumnRenamed(col, col.lower())

In [29]:
df.createOrReplaceTempView('df')

### CSV para PARQUET

In [30]:
df.write.parquet("MICRODADOS_IES_CONSULTA.parquet")


In [31]:
parquet_df = spark.read.parquet("MICRODADOS_IES_CONSULTA.parquet")

In [32]:
parquet_df.createOrReplaceTempView("parquet_df")

### Consultas


### Olhando para o estado de Minas Gerais, quantos municípios têm informações presentes na base de dados?


In [41]:
a1 = spark.sql("""
select COUNT(DISTINCT no_municipio_ies) from parquet_df
where sg_uf_ies = 'MG'
""")
a1.show()


+--------------------------------+
|count(DISTINCT no_municipio_ies)|
+--------------------------------+
|                             104|
+--------------------------------+



### Quantos professores doutores existem em cada cidade de Minas Gerais presente na base de dados

In [42]:
a2 = spark.sql("""SELECT no_municipio_ies as municipio, sum(qt_doc_ex_dout) as Qtd_Doutores from parquet_df WHERE no_uf_ies = 'Minas Gerais' 
          group by no_municipio_ies order by Qtd_Doutores DESC""")
a2.show()


+--------------------+------------+
|           municipio|Qtd_Doutores|
+--------------------+------------+
|      Belo Horizonte|        6615|
|          Uberl�ndia|        1939|
|        Juiz de Fora|        1915|
|              Vi�osa|        1185|
|       Montes Claros|         925|
|             Uberaba|         891|
|              Lavras|         812|
|    S�o Jo�o del Rei|         753|
|          Ouro Preto|         746|
|          Diamantina|         673|
|             Alfenas|         661|
|        Pouso Alegre|         499|
|             Itajub�|         304|
|            Contagem|         219|
|         Sete Lagoas|         201|
|               Betim|         192|
|         Divin�polis|         133|
|        Bom Despacho|         129|
|Governador Valadares|         110|
|            Ipatinga|         107|
+--------------------+------------+
only showing top 20 rows



### Qual a quantidade de docentes com deficiência no estado do Paraná?

In [43]:
a3 = spark.sql("SELECT no_municipio_ies, qt_doc_ex_dout FROM parquet_df WHERE sg_uf_ies = 'MG' ")
a3.show()


+--------------------+--------------+
|    no_municipio_ies|qt_doc_ex_dout|
+--------------------+--------------+
|          Ouro Preto|           746|
|              Vi�osa|          1123|
|          Uberl�ndia|          1702|
|       Tr�s Cora��es|            45|
|             Alfenas|           162|
|Conselheiro Lafaiete|            12|
|    S�o Jo�o del Rei|           722|
|Santa Rita do Sap...|            21|
|             Uberaba|             0|
|            Varginha|             1|
|          Uberl�ndia|            36|
|             Uberaba|           142|
|Santa Rita do Sap...|            10|
|        Pouso Alegre|            12|
|      Belo Horizonte|             0|
|      Belo Horizonte|            11|
|             Itajub�|            20|
|Governador Valadares|             8|
|           Barbacena|            23|
|           Barbacena|            45|
+--------------------+--------------+
only showing top 20 rows

